In [71]:
import folium

map = folium.Map(location=[40.71552394800378, -74.01878975440212], zoom_start=11, tiles='CartoDB positron')

borough_layer = folium.FeatureGroup(name='Boroughs')
score_layer = folium.FeatureGroup(name='Score Layer')

In [72]:
import pandas as pd

df = pd.read_csv('/Users/loic.doerr/dev/ny-emergency-ai/borough_data.csv')

df.head()

,Borough,Population,Area,Population Density,Firehouse Quantity,Firehouse Density (Population),Firehouse Density (Area),Population Density Transformed,Firehouse Density (Population) Transformed,Firehouse Density (Area) Transformed,Score
0,Bronx,1446788,1.187175e+09,121.868156,34,2.350033,0.002864,0.404107,0.084234,0.258752,0.747093
1,Brooklyn,2648452,1.934138e+09,136.931889,66,2.492022,0.003412,0.470372,0.158306,0.345698,0.974375
2,Manhattan,1638281,6.366456e+08,257.330149,48,2.929900,0.007540,1.000000,0.386736,1.000000,2.386736
3,Queens,2330295,3.041419e+09,76.618689,51,2.188564,0.001677,0.205056,0.000000,0.070554,0.275610
4,Staten Island,487155,1.623621e+09,30.004236,20,4.105470,0.001232,0.000000,1.000000,0.000000,1.000000


In [73]:
import json

with open('./data/geo-data/boroughs.geojson') as rawData:
  boroughs_boundaries_geo = json.load(rawData)

In [74]:
folium.GeoJson(
  boroughs_boundaries_geo,
  style_function=lambda feature: {
    'fillColor': 'green' if feature['properties']['boro_name'] == 'Manhattan' else 'blue' if feature['properties']['boro_name'] == 'Brooklyn' else 'red' if feature['properties']['boro_name'] == 'Bronx' else 'orange' if feature['properties']['boro_name'] == 'Queens' else 'purple',
    'color': 'black',
    'weight': 2,
    'fillOpacity': 0.2,
  }
).add_to(borough_layer)

In [75]:
borough_population_density = dict(zip(df['Borough'], df['Score']))

population_density_values = list(borough_population_density.values())

max_density = max(population_density_values)

def normalize(value, min_value, max_value):
  return (value - min_value) / (max_value - min_value)

folium.GeoJson(
  boroughs_boundaries_geo,
  style_function=lambda feature: {
    'fillColor': 'red',
    'color': 'black',
    'weight': 2,
    'fillOpacity': normalize(
      borough_population_density.get(feature['properties']['boro_name'], 0), 
      0, 
      max_density
    ),
  }
).add_to(score_layer)

In [76]:
map.add_child(borough_layer)
map.add_child(score_layer)

folium.LayerControl().add_to(map)

map.save('./maps/hallo.html')